In [1]:
import pandas as pd
from scipy.stats import mannwhitneyu
from pathlib import Path

# -------------------------------
# User Inputs
# -------------------------------
file_path = r"D:\climate change\Tutorial_Climate Data_2025-26.xlsx"
sheet_name = "Rainfall"  # change sheet
split_year = 1995
output_file = r"D:\climate change\Mann_Whitney_Results.xlsx"

# -------------------------------
# Load data (skip Lon/Lat rows)
# -------------------------------
print(f"Loading data from {file_path}, sheet '{sheet_name}'...")
try:
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=2)
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {file_path}")
    exit()
except Exception as e:
    print(f"❌ ERROR: Could not read Excel file. {e}")
    exit()

# Rename first column to Date
df = df.rename(columns={df.columns[0]: "Date"})

# Convert to date
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
df["Year"] = df["Date"].dt.year

# Identify station columns (everything except Date and Year)
value_columns = [c for c in df.columns if c not in ["Date", "Year"]]
print(f"Found {len(value_columns)} station columns to analyze.")

# -------------------------------
# Mann-Whitney U Test
# -------------------------------
print(f"Performing Mann-Whitney U test split at year {split_year}...")
results = []

for col in value_columns:
    pre = df[df["Year"] <= split_year][col].dropna()
    post = df[df["Year"] > split_year][col].dropna()

    if len(pre) > 0 and len(post) > 0:
        U, p = mannwhitneyu(pre, post, alternative="two-sided")
        conclusion = "Significant Change" if p < 0.05 else "No Significant Change"
        
        # Round the numeric results for output
        U_rounded = round(U, 2)
        p_rounded = round(p, 2)
        
    else:
        U_rounded, p_rounded, conclusion = None, None, "Not Enough Data"

    results.append({
        "Station": col,
        "U Statistic": U_rounded,
        "p-value": p_rounded,
        "Conclusion": conclusion
    })

# Save results
try:
    pd.DataFrame(results).to_excel(output_file, index=False)
    print(f"\n✅ Mann-Whitney test completed.\n📁 Saved to: {output_file}")
except Exception as e:
    print(f"\n❌ ERROR: Could not save results to {output_file}. {e}")


Loading data from D:\climate change\Tutorial_Climate Data_2025-26.xlsx, sheet 'Rainfall'...
Found 23 station columns to analyze.
Performing Mann-Whitney U test split at year 1995...

✅ Mann-Whitney test completed.
📁 Saved to: D:\climate change\Mann_Whitney_Results.xlsx
